<a href="https://colab.research.google.com/github/belladu0201/AIPI531_Deep-Reinforcement-Learning-and-Applications_SP23/blob/main/AIPI531_HW1_notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 531: Deep Reinforcement Learning and Applications (SPRING 23)
Name: Beibei Du


## Question 2 (AIPI 531) Train an agent to play any game/task in SB3 using A2C with Colab
### Requirements:
Pick a task/game in SB3.
- Add a TensorBoard to visualize the training curves.
- Include/record the final evaluation video.
- Saving/loading the policy/model.
- Submit a link to the Colab notebook including the
training/testing experiments and results.

In [29]:
# sync with drive
# from google.colab import drive
# drive.mount('/content/drive')


In [30]:
# step4 in the setup
import os
os.chdir('/content/drive/My Drive/AIPI531/stable-baselines3')
# !ls


In [31]:
%cd /content/drive/My Drive/AIPI531/stable-baselines3

/content


In [32]:
!pip install -e .


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/Othercomputers/My%20MacBook%20Pro/AIPI531/stable-baselines3
  Preparing metadata (setup.py) ... done
  Attempting uninstall: stable-baselines3
    Found existing installation: stable-baselines3 1.8.0a4
    Can't uninstall 'stable-baselines3'. No files were found to uninstall.
  Running setup.py develop for stable-baselines3


In [33]:
import stable_baselines3


In [34]:
 # !pip install git "https://github.com/belladu0201"

In [35]:
# pip install stable-baselines3[extra]

In [36]:
# !apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
# !pip install stable-baselines3[extra] pyglet==1.5.27

In [37]:
# import stable_baselines3
stable_baselines3.__version__

'1.8.0a4'

In [38]:
import gym
import numpy as np

In [39]:
# import RL model
from stable_baselines3 import PPO

In [40]:
# Imort Policy Class
from stable_baselines3.ppo import MlpPolicy

I make the environment as a Taxi Game environment

In [41]:
env = gym.make('Taxi-v3')

model = PPO(MlpPolicy, env, verbose=0)

In [42]:
def evaluate(model, num_episodes=100, deterministic=True):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs, deterministic=deterministic)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [43]:
from stable_baselines3.common.evaluation import evaluate_policy

In [44]:
# Use a separate environement for evaluation
eval_env = gym.make('Taxi-v3')

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/content/drive/Othercomputers/My MacBook Pro/AIPI531/stable-baselines3/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:-1366.49 +/- 855.99


In [45]:
!pip install tensorboard
! pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [47]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## The following is the training process

In [48]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)

In [49]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:-200.00 +/- 0.00


In [50]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [51]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [52]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make('Taxi-v3')])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [53]:
record_video('Taxi-v3', model, video_length=500, prefix='ppo-cartpole')

Saving video to /content/drive/Othercomputers/My MacBook Pro/AIPI531/stable-baselines3/videos/ppo-cartpole-step-0-to-step-500.json


In [54]:
show_videos('videos', prefix='ppo')

In [55]:
model = PPO('MlpPolicy', "Taxi-v3", verbose=1).learn(1000)

Using cpu device
Creating environment from the given name 'Taxi-v3'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 192      |
|    ep_rew_mean     | -735     |
| time/              |          |
|    fps             | 1226     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
